# TILT - Teemo Induced Loss of Tranquility
A research insentive to study main factors in inducing tilt.


# Imports

In [1]:
# from collections import defaultdict
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError


# import arrow
# import csv
import itertools
# import datetime
# import time
import json
import numpy as np
import pandas as pd

# Functions

In [2]:
  
def api_key(api_key_loc):
    """
    Read in the development API key from a file and checks if it is viable.
    
    Args:
        credentials (str): Name of json file containing the credentials.
    Returns:
        api_key (str): The API key.
    """

    while True:
        with open(api_key_loc, "r") as json_data:
            creds = json.load(json_data)
            api_key = creds["dev_api_key"]
            lol_watcher = LolWatcher(creds["dev_api_key"])
            try:
                # Validate API key by using it to check server status
                lol_watcher.lol_status.shard_data("euw1")
                # Break if key is functional
                break
            except ApiError as error:
                # If the current API key does not work input new one
                if error.response.status_code == 403:
                    new_api_key = input("API key is incorrect, enter correct key here.")
                    creds["dev_api_key"] = new_api_key
                    # Replace the old API key
                    with open(api_key_loc, "w") as json_data:
                        json.dump(creds, json_data)
    return api_key

In [3]:
def trans_reg(reg_abbrv):
    """
    Translate a league of legends region into a region readable by riot watcher.
    
    Args:
        reg_abbrv (str): Abbreviation of a official registered Riot servers that
                         hosts league of legends (e.g. euw1).
    Returns:
        rw_region (str): Riot Watcher region the reg_abbrv server falls under.
    """
    # Look up in the list what the riot watcher region is for the given region abbreviation
    regions_metadata = {"br1": "americas",
                        "eun1": "europe",
                        "euw1": "europe",
                        "jp1": "asia",
                        "kr": "asia",
                        "la1": "americas",
                        "la2": "americas",
                        "na1": "americas",
                        "oc1": "americas",
                        "tr1": "europe",
                        "ru": "europe"
                        }

    for reg, rw_reg in regions_metadata.items():
        if reg_abbrv.lower() == reg:
            rw_region = rw_reg
    
    return rw_region

In [4]:
def get_matches(reg, pid, s_time, e_time):
    """
    Retrieve match IDs from a player within a given timeframe.

    Args:
        reg (str): Abbreviation of a official registered Riot servers that
                   hosts league of legends (e.g. euw1).
        pid (str): An Encrypted globally unique identifyer for a player.
        s_time (long): Start of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        e_time (long): end of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
    Returns:
        matches (list): League of legends match IDs in chronological order
                        starting with the most recent match.
    """

    # Retrieve all match ID's between two time points 
    matches = lol_watcher.match.matchlist_by_puuid(region=trans_reg(reg),
                                                   puuid=pid,
                                                   start_time=s_time,
                                                   end_time=e_time,
                                                   count=100)
    
    # Geather more match IDs in case matches exceeds the standard limit of 100
    while len(matches) % 100 == 0 and len(matches) != 0:
        # Geather match details of earliest match
        match_deets = lol_watcher.match.by_id(trans_reg(reg), matches[-1])
        # Start time of earliest match
        early_g_start = int(str(match_deets["info"]['gameCreation'])[:10])
        # Select 100 matches previous to early_g_start  
        match_add = lol_watcher.match.matchlist_by_puuid(region=trans_reg(reg),
                                                         puuid=pid,
                                                         start_time=s_time,
                                                         end_time=early_g_start,
                                                         count=100)
        if len(match_add) == 0:
            break
        else:
            matches.extend(match_add)

    return matches

In [5]:
def geather_data(puuid, matches):
    """
    Geather player data from a match and chronology data of the match.
    
    Args:
        puuid (str): An Encrypted globally unique identifyer for a player.
        matches (list): League of legends match IDs in chronological order
                        starting with the most recent matches.  
    Returns:
        df_matches_data (df): Data geathered from matches of a player.
    """
    
    time_info = ["gameCreation", "gameStartTimestamp", "gameDuration"]
    keep_data = ["individualPosition", "role", "teamPosition", "lane", "kills", "assists", "deaths", 
                 "doubleKills", "tripleKills", "quadraKills", "pentaKills", 
                 "killingSprees", "largestKillingSpree", 
                 "teamEarlySurrendered", "gameEndedInEarlySurrender", "gameEndedInSurrender",
                 "neutralMinionsKilled", "totalMinionsKilled", "teamId", "win"]

    col_names = ["match_id", "indiv_pos", "role", "team_pos", "lane",
                 "kills", "assists", "deaths", 
                 "2_kills", "3_kills", "4_kills", "5_kills", 
                 "kill_spree", "max_kill_spree", 
                 "early_surr_try", "early_surr", "game_surr",
                 "neutral_kills", "mini_kills", "team_id", "win",
                 "game_make", "game_start", "game_dur"]
      
    comp_data = []
    # Geather match details
    for match_id in matches:
        reg = match_id.split("_")[0].lower()
        match_deets = lol_watcher.match.by_id(trans_reg(reg), match_id)        
        # Time data
        time_data = dict((key, match_deets["info"][key]) for key in time_info)

        # Player data       
        for part_info in match_deets["info"]["participants"]:
            if part_info["puuid"] == puuid:
                filt_data = dict((key, part_info[key]) for key in keep_data)
        match_data = {"match_id": match_id} | filt_data | time_data
        comp_data.append(match_data.values())
    
    # Store all data in a dataframe
    df_matches_data = pd.DataFrame(comp_data, columns=col_names)

    return df_matches_data
    

In [6]:
def filt_matches(matches_data, max_rest, min_streak):
    """
    Filter matches based on rest time in between matches and number of matches played in a row.  
    
    Args:
        matches_data (df): Data geathered from matches of a player
        max_rest (int): Rest time in between matches in miliseconds e.g. 3600000 = 1 hour.
        min_streak (int): Minimum games played in a row with less then max_rest between.
    Returns:
        df_match_filt (df): Matches filtered based on minuum subsequent matches within
                            the maximum rest time in between the matches.
    """
      
    # Filter for matches based on rest time

    # Add rest time
    df_match_filt = matches_data 
    df_match_filt["game_end"] = df_match_filt.loc[:,["game_start", "game_dur"]].sum(axis=1)
    df_match_filt["prev_game_make"] = df_match_filt["game_make"].shift(-1, fill_value=0)
    df_match_filt["rest_time"] = df_match_filt["game_end"] - df_match_filt["prev_game_make"]

    # Store if the game ID should be kept if rest time is below max_rest
    df_match_filt['keep'] = df_match_filt["rest_time"] <= max_rest
    # Grouping based on matches played subsequently 
    subseq_true = df_match_filt['keep'].diff().ne(0).cumsum()
    # Drop streaks that are lower then min_streak
    df_match_filt = df_match_filt[df_match_filt['keep'].groupby(subseq_true).transform("count") >= min_streak]
    # Drop matches with rest times above max_rest 
    df_match_filt = df_match_filt[~(df_match_filt["keep"] == 0.0)]

    # Add the streak id to the df_match_filt
    df_match_filt["streak_id"] = df_match_filt.groupby(subseq_true).grouper.group_info[0]

    # Remove the keep column 
    df_match_filt.drop("keep", axis=1, inplace = True)
    
    return df_match_filt


In [7]:

def get_summoners(regs, tiers, divs, sum_lim, s_time, e_time, max_rest, min_streak):
    """
    Get puuid's, tier and division, from all regions that are in ranks below master rank ((random?) above bronze?).
    
    Args:
        regs (list): Official registered Riot servers that hosts league of legends.
        tier (list): Tiers below Master rank.
        divs (list): Divisions in roman numerals.
        sum_lim (int): Maximum number of summoner ids to collect per region, tier and division
        s_time (long): Start of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        e_time (long): end of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        max_rest (int): Rest time in between matches in miliseconds e.g. 3600000 = 1 hour.
        min_streak (int): Minimum games played in a row with less then max_rest between.
    Returns:
        sumo_info (list): Summoners puuid, region, tier, and division.
    """
    
    summs_info=[]
    # for reg, tier, div in itertools.product(regs, tiers, divs): # real line change what we want in teh settings file
    for reg, tier, div in itertools.product(regs[0:1], tiers[1:2], divs[0:1]): # test line
        # print(reg, tier, div) # TODO: remove me later
        summs_div = 0
        page_nr = 0
        # Keep adding new summoners until the summoner limit has been reached
        while summs_div < sum_lim:
            page_nr += 1
            summs = lol_watcher.league.entries(reg, "RANKED_SOLO_5x5", tier, div, page_nr)
            # Look into data per summoner
            for sumo in summs:
                if summs_div < sum_lim:
                    # Get puuid
                    pid = lol_watcher.summoner.by_id(reg, sumo["summonerId"])["puuid"]

                    # Retrieve all match ID's between two time points
                    match_ids = get_matches(reg, pid, s_time, e_time)
                    # Skip players that have no matches in the given time frame 
                    if not match_ids:
                        continue
        
                    # Geather data of the player
                    match_info = geather_data(pid, match_ids)
                    
                    # Filter matches
                    # TODO: these things you filter on should be in settings 
                    filt_match_info = filt_matches(match_info, max_rest, min_streak)
                    # Skip players that have no matches left after filtering
                    if filt_match_info.empty:
                        continue
                    
                    # Store the info 
                    # TODO: Think of a smart dictionary/ list / dataframe solution for this
                    summs_info.append([pid, reg, tier, div, filt_match_info]) # add time and change len to actual matches
                    summs_div += 1
            if summs_div == sum_lim:
                break

    return(summs_info)


# Main

In [9]:
# Directory locations

# Project folders
proj_dir = Path.cwd().parent

# Raw data storage
data_dir = proj_dir / "data"

# Out dir
out_dir = proj_dir / "out"

# Gobal variables

# Set API key
api_key_loc = data_dir / "dev_api_key.json"

# Enter API key
lol_watcher = LolWatcher(api_key(api_key_loc))

# Read user settings 
settings_loc = proj_dir / "settings" / "config.json"
with open(settings_loc, "r") as settings_data:
    settings = json.load(settings_data)


### Collect summoners

In [10]:
# Retrieve the summoner info of X players
# move start end time to settings
summs_info = get_summoners(regs=settings["regions"],
                           tiers=settings["tiers"],
                           divs=settings['divisions'],
                           sum_lim=settings['summoner_limit'],
                           s_time=settings["start_time"],
                           e_time=settings["end_time"],
                           max_rest=settings["max_rest"],
                           min_streak=settings["min_streak"])

print(summs_info)

[['Elq5G_lCAVM4MKyDuwso7zKqIO6vsPWbRoXCriT6QAddOEWyFyzl5gdL1f5ew4YmvUJDHTGhwP4tGg', 'eun1', 'BRONZE', 'I',            match_id indiv_pos     role team_pos    lane  kills  assists  \
9   EUN1_2869053241    BOTTOM    CARRY   BOTTOM  BOTTOM     15        9   
10  EUN1_2869020552    BOTTOM    CARRY   BOTTOM  BOTTOM      4        4   
11  EUN1_2869077855   Invalid  SUPPORT             NONE      6       10   
27  EUN1_2863508959   Invalid      DUO              TOP     12       24   
28  EUN1_2863425509   Invalid  SUPPORT           MIDDLE     14       30   
29  EUN1_2863372889   Invalid  SUPPORT             NONE     12       16   
30  EUN1_2863429238   Invalid      DUO             NONE     11       11   
31  EUN1_2863306936   Invalid  SUPPORT             NONE      8       17   
33  EUN1_2863041835   Invalid  SUPPORT           MIDDLE      3       28   
34  EUN1_2863040155   Invalid  SUPPORT             NONE      2       24   
35  EUN1_2863029373   Invalid  SUPPORT             NONE      7      

### User info
Some simple scripts to retrieve basic information of the user not really needed just for quick test

In [ ]:
# short script to IO files

summ_list_loc = out_dir / "summoner_list.tsv"
with open(summ_list_loc, "w") as out_file:    
    sumo_info.to_csv(out_file, sep='\t', index = False, line_terminator='\n')


# Read in the summoner information
sumo_info = pd.read_csv(summ_list_loc, sep = "\t")
